In [1]:
%pip install requests
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Google code
# %load_ext autoreload
# %autoreload 2

In [1]:
import numpy as np
import pandas as pd
import proballer as pb
import requests
import sys
from bs4 import BeautifulSoup

In [4]:
# Google code
# authenticate
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Google code
# sys.path.append('/content/drive/My Drive/Colab Notebooks')

## Players

In [13]:
# Google code
# df_players = pd.read_excel('/content/drive/My Drive/Colab Notebooks/aus_boomers_dev.xlsx')
# df_players = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AusBoomers.xlsx')

In [4]:
df_players = pd.read_excel('../Data/aus_boomers_players.xlsx')
df_players.head()

In [6]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Name         25 non-null     object        
 1   ProballerID  25 non-null     int64         
 2   Position     25 non-null     object        
 3   Shoots       16 non-null     object        
 4   Height       25 non-null     int64         
 5   Weight       13 non-null     float64       
 6   DOB          25 non-null     datetime64[ns]
 7   Selected     13 non-null     object        
 8   G            13 non-null     float64       
 9   PTS          13 non-null     float64       
 10  TRB          13 non-null     float64       
 11  AST          13 non-null     float64       
 12  FG%          13 non-null     float64       
 13  FG3%         13 non-null     float64       
 14  FT%          13 non-null     float64       
 15  eFG%         13 non-null     float64       
 16  PER       

In [9]:
# Filter players to final team selection
df_players = df_players.loc[df_players['Selected'] == 'Y']

# Select first 7 rows
df_players = df_players.iloc[:, :7]

In [10]:
df_players

,Name,ProballerID,Position,Shoots,Height,Weight,DOB
2,Patty Mills,47394,PG,R,185,81.0,1988-08-11
3,Aron Baynes,37144,C,R,208,117.0,1986-12-09
4,Joe Ingles,47393,SF,L,203,NaN,1987-10-02
7,Matthew Dellavedova,49126,PG,R,193,NaN,1990-09-08
8,Matisse Thybulle,178760,SG,R,196,NaN,1997-03-04
9,Dante Exum,56767,PG,R,196,97.0,1995-07-13
10,Josh Green,174018,SG,R,196,90.0,2000-11-16
13,Chris Goulding,48598,SG,NaN,192,92.0,1988-10-24
14,Nick Kay,69040,C,NaN,205,NaN,1992-08-03
15,Jock Landale,72804,C,NaN,213,NaN,1995-10-25


## Methodology

Each player has an id.  Each page has one or more tables.  

- First table is 'regular' season.  
- Subsequent tables for playoff, European, or national games, are in tabbed panels.

In [11]:
df0 = pd.DataFrame()

for index, row in df_players.iterrows():

  id = row['ProballerID']
  name = row['Name']

  url = pb.get_player_url(str(id),name)

  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser') 

  # first pass
  df1 = pb.get_primary_table(soup,name)

  # second pass
  df2 = pb.get_secondary_tables_by_class(soup,name)

  df0 = df0.append(df1,ignore_index=True)
  df0 = df0.append(df2,ignore_index=True)





--- https://www.proballers.com/basketball/player/47394/patty-mills ---
list-regular, 4
--- https://www.proballers.com/basketball/player/37144/aron-baynes ---
list-regular, 5
--- https://www.proballers.com/basketball/player/47393/joe-ingles ---
list-regular, 5
--- https://www.proballers.com/basketball/player/49126/matthew-dellavedova ---
list-regular, 4
--- https://www.proballers.com/basketball/player/178760/matisse-thybulle ---
list-regular, 4
--- https://www.proballers.com/basketball/player/56767/dante-exum ---
list-regular, 4
--- https://www.proballers.com/basketball/player/174018/josh-green ---
list-regular, 4
--- https://www.proballers.com/basketball/player/48598/chris-goulding ---
list-regular, 5
--- https://www.proballers.com/basketball/player/69040/nick-kay ---
list-regular, 4
--- https://www.proballers.com/basketball/player/72804/jock-landale ---
list-regular, 5
--- https://www.proballers.com/basketball/player/176811/duop-reath ---
list-regular, 5
--- https://www.proballers.com

In [12]:
# check for non-numeric columns
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Day       65 non-null     float64
 1   OPPONENT  65 non-null     object 
 2   League    65 non-null     object 
 3   SCORE     65 non-null     object 
 4   Pts       65 non-null     float64
 5   Reb       65 non-null     float64
 6   Ast       65 non-null     float64
 7   MIN       65 non-null     float64
 8   2M-2A     65 non-null     object 
 9   3M-3A     65 non-null     object 
 10  FG%       65 non-null     object 
 11  1M-1A     65 non-null     object 
 12  1%        65 non-null     object 
 13  Or        65 non-null     float64
 14  Dr        65 non-null     float64
 15  Stl       65 non-null     float64
 16  To        65 non-null     float64
 17  Blk       65 non-null     float64
 18  Eff       65 non-null     float64
 19  Source    65 non-null     object 
 20  Name      65 non-null     object 


In [13]:
# replace character with NaN
df = df0.replace('-',np.nan)

In [19]:
df.head()

,Day,OPPONENT,League,SCORE,Pts,Reb,Ast,MIN,2M-2A,3M-3A,...,1M-1A,1%,Or,Dr,Stl,To,Blk,Eff,Source,Name
0,20210807.0,@ Slovenia,Olympics,W 93-107,42.0,3.0,9.0,38.0,11-16,4-15,...,8-10,80.0%,0.0,3.0,1.0,2.0,0.0,35.0,list-regular,Patty Mills
1,20210805.0,@ USA,Olympics,L 97-78,15.0,5.0,8.0,29.0,2-9,3-5,...,2-2,100.0%,3.0,2.0,3.0,2.0,0.0,20.0,list-regular,Patty Mills
2,20210803.0,vs Argentina,Olympics,W 97-59,18.0,2.0,4.0,25.0,4-11,3-4,...,1-2,50.0%,2.0,0.0,1.0,2.0,0.0,14.0,list-regular,Patty Mills
3,20210731.0,vs Germany,Olympics,W 89-76,24.0,1.0,6.0,33.0,4-8,5-15,...,1-1,100.0%,0.0,1.0,0.0,3.0,0.0,14.0,list-regular,Patty Mills
4,20210728.0,@ Italy,Olympics,W 83-86,16.0,6.0,5.0,36.0,4-8,2-9,...,2-2,100.0%,1.0,5.0,1.0,2.0,0.0,15.0,list-regular,Patty Mills


In [20]:
df.to_excel('../Data/aus_boomers_proballer_selected.xlsx',index=False)